In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import ShuffleSplit
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import pylab
import random
from sklearn.model_selection import KFold
import plotly.express as px
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestRegressor
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
from  sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.svm import SVR
import dateutil
from datetime import datetime
from dateutil.relativedelta import relativedelta
from mlxtend.evaluate import PredefinedHoldoutSplit
import kneed
import kaleido
%run functions.ipynb

In [ ]:
%%notify
%%time

X = np.array(maual_train[list_feat])
y = np.array(maual_train[[predict]])

split_algo = ShuffleSplit(test_size=0.20, random_state=0)
cv_gen = split_algo.split(X, y)
cv = list(cv_gen)

sfs = SFS(algo,
          k_features=(len(list_feat)),
          forward=True,
          floating=False, 
          scoring= scorer_rmse, 
          cv=cv)
sfs = sfs.fit(X, y.ravel())

fig = plot_sfs(sfs.get_metric_dict(), kind='std_err')
plt.title("RMSE %s %s" % (predict, station))
plt.grid()
plt.show()

In [ ]:
station = "erlabrunn"

In [ ]:
#OPTIONS:  "OPO4P", "NO3N", "NH4N", "TRP"


In [ ]:
if station == "kahl":
    predict = "NH4N"
    df1 = pd.read_csv('%s_%s_final.csv' %(station, predict))
    df1['Datum'] = pd.to_datetime(df1['Datum'])
    df1= df1.set_index("Datum")
    df1.drop(del_columns, inplace=True, axis=1)
    df1 = df1.rename({'O2':('O2_%s'%(predict)), 'Temp':('Temp_%s'%(predict)), 'pH':('pH_%s'%(predict)), 
                      'flow':('flow_%s'%(predict)), 'Conduct':('Conduct_%s'%(predict))}, axis=1) 

In [ ]:
df1

In [ ]:
predict = "NO3N"
df2 = pd.read_csv('%s_%s_final.csv' %(station, predict))
df2['Datum'] = pd.to_datetime(df2['Datum'])
df2= df2.set_index("Datum")
df2.drop(del_columns, inplace=True, axis=1)
df2 = df2.rename({'O2':('O2_%s'%(predict)), 'Temp':('Temp_%s'%(predict)), 'pH':('pH_%s'%(predict)), 
                  'flow':('flow_%s'%(predict)), 'Conduct':('Conduct_%s'%(predict))}, axis=1) 
df2

In [ ]:
df2.describe()

In [ ]:
predict = "OPO4P"
df3 = pd.read_csv('%s_%s_final.csv' %(station, predict))
df3['Datum'] = pd.to_datetime(df3['Datum'])
df3= df3.set_index("Datum")
df3.drop(del_columns, inplace=True, axis=1)
df3 = df3.rename({'O2':('O2_%s'%(predict)), 'Temp':('Temp_%s'%(predict)), 'pH':('pH_%s'%(predict)), 
                  'flow':('flow_%s'%(predict)), 'Conduct':('Conduct_%s'%(predict))}, axis=1) 
df3

In [ ]:
if station == "kahl":
    df = pd.concat([df1,df2,df3], axis=1)
if station == "erlabrunn":
    df = pd.concat([df2,df3], axis=1)
    
df

In [ ]:
if station == "kahl":
    df['O2_NO3N'].update(df.pop('O2_NH4N'))
    df['O2_NO3N'].update(df.pop('O2_OPO4P'))

    df['Temp_NO3N'].update(df.pop('Temp_NH4N'))
    df['Temp_NO3N'].update(df.pop('Temp_OPO4P'))

    df['pH_NO3N'].update(df.pop('pH_NH4N'))
    df['pH_NO3N'].update(df.pop('pH_OPO4P'))

    df['flow_NO3N'].update(df.pop('flow_NH4N'))
    df['flow_NO3N'].update(df.pop('flow_OPO4P'))

    df['Conduct_NO3N'].update(df.pop('Conduct_NH4N'))
    df['Conduct_NO3N'].update(df.pop('Conduct_OPO4P'))

df

In [ ]:
if station == "erlabrunn":
    df['O2_NO3N'].update(df.pop('O2_OPO4P'))

    df['Temp_NO3N'].update(df.pop('Temp_OPO4P'))

    df['pH_NO3N'].update(df.pop('pH_OPO4P'))

    df['flow_NO3N'].update(df.pop('flow_OPO4P'))

    df['Conduct_NO3N'].update(df.pop('Conduct_OPO4P'))

In [ ]:
df.describe()

In [ ]:
figx = px.line(df, x = df.index, y='pH_NO3N', title = "%s " %(station))
# figx = px.line(df, x = df.index, y=['O2_NH4N', "Predicted_%s" %(predict)], title = "%s " %(station))
figx.update_xaxes(rangeslider_visible = True)
figx.show()

In [ ]:
if linear_regression == 1:
    temp_feats = []
    main_adj_r2 = []
    main_rmse = []

    # corr_order = ["Cube_Conduct", "pH", "log_Turb", "log_Chlorophyll", "log_flow", "log_O2", "Temp"]
    # corr_order = ["Temp"]

    for i in corr_order:
        temp_feats.append(i)
        if k_fold == 1:

            X = np.array(major_X[temp_feats])
            y = np.array(major_y)

            arr_Adj_r2 = []
            arr_RMSE = []

            kf = KFold(n_splits=5)
            for train_index, test_index in kf.split(X):
                X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
                linear = linear_model.LinearRegression()
                linear.fit(X_train, y_train)
                predictions = linear.predict(X_test)
                pd_predictions = pd.DataFrame(predictions, columns=["predicted"])
                pd_observed = pd.DataFrame(y_test, columns=["observed"])
                pd_concat = pd.concat([pd_observed, pd_predictions], axis=1)

                temp = Adj_r2(y_test, predictions, X_test)
                arr_Adj_r2.append(temp)


                temp2 = mean_squared_error(y_test, predictions, squared=False)
                arr_RMSE.append(temp2)
    #             print(arr_RMSE)



            adj_r2 = np.average(arr_Adj_r2)
            main_adj_r2.append(adj_r2)
            rmse = np.average(arr_RMSE)
            main_rmse.append(rmse)

        #         print("Cross Validation results \n")
        #         print("Adjusted Rsquared", predict, "\n", adj_r2)
        #         print('RMSE ', predict, '\n', rmse)

        elif k_fold == 0:
            O_features = features[temp_feats]
            X_train = np.array(O_features.loc[:train_end])
            y_train = np.array(label.loc[:train_end])

            X_test = np.array(O_features.loc[test_start:])
            y_test = np.array(label.loc[test_start:])

            linear = linear_model.LinearRegression()
            linear.fit(X_train, y_train)
            predictions = linear.predict(X_test)

            rmse = mean_squared_error(y_test, predictions, squared=False)
            adj_r2 = Adj_r2(y_test, predictions, X_test)
            print("Without Cross Validation \n")
            print("Adjusted Rsquared", predict, "\n", adj_r2)
            print('RMSE ', predict, '\n', rmse)

In [ ]:
def std_procedure(filename):
    df = pd.read_csv(filename)
    df['Datum'] = pd.to_datetime(df['Datum'])
    df= df.set_index("Datum")
    return df

def export(df, filename):
    df.to_csv('%s_new.csv' %filename, index = True, header = True)

In [ ]:
figx = px.line(df, x = df.index, y='O2_NO3N', title = "%s " %(station))
# figx = px.line(df, x = df.index, y=['O2_NH4N', "Predicted_%s" %(predict)], title = "%s " %(station))
figx.update_xaxes(rangeslider_visible = True)
figx.show()

In [ ]:
def order_of_features(df, dict):
    df = df[["feature_names"]]
    rows = len(df.index)
    array = []
    i =1
    while i <= rows:
        test = df.iloc[(i-1),0]
        array.append(test)
        i = i+1

    distinct = []
    for i in array:
        for j in i:
            distinct.append(j)

    final_distinct = list(dict.fromkeys(distinct))
    
    final_array = []
    for i in final_distinct:
        final_array.append(dict.get(i))
        
    return final_array

In [ ]:
def column_dict(df):
    ss = df.columns
    dict = {}
    i = 0
    while i < len(ss):
        dict[str(i)] = ss[i]
        i = i+1
    return dict

ss = column_dict(tryy)
ss

In [ ]:
i = 0
while i < len(post_count):
    percentage_loss = (pre_count[i] - post_count[i])/pre_count[i] * 100
    print("Percentage loss in ", columns[i], ": ", percentage_loss, "\n")
    i = i+1

In [ ]:
post_count = []

for i in columns:
    temp = df[[i]]
    count = post_total_points - np.count_nonzero(temp.isnull())
    post_count.append(count)

In [ ]:
pre_count = []

for i in columns:
    temp = df[[i]]
    count = pre_total_points - (np.count_nonzero(temp.isnull()))
    pre_count.append(count)

print(pre_count)

In [ ]:
post_total_points = len(df.index)
post_total_points

In [ ]:
def r2_score(y_test, predictions):
    corr_matrix = np.corrcoef(y_test, predictions)
    corr = corr_matrix[0,1]
    R_sq = corr**2
    return R_sq

In [ ]:
pd_y_test = pd.DataFrame(y_test)
pd_y_test.columns = ["y_test"]
pd_y_test_mean = pd_y_test.mean()
# pd_y_test

pd_predictions = pd.DataFrame(predictions)
pd_predictions.columns = ["predictions"]
pd_predictions = pd_predictions.round(2)
pd_predictions_mean = pd_predictions.mean()
# pd_predictions

dff = pd.concat([pd_predictions, pd_y_test], axis=1)
# dff

dff["numerator_NSE"] = pow(dff["predictions"] - dff["y_test"],2)
dff["denominator_NSE"] = pow(pd_y_test - pd_y_test_mean,2)
# dff

dff["X1"]= (pd_y_test - pd_y_test_mean)
dff["X2"]=(pd_predictions - pd_predictions_mean)
# dff

dff["numerator_Rsquared"] = dff["X1"]*dff["X2"]
# dff

dff["denominator_2"] = pow(pd_predictions - pd_predictions_mean,2)
# dff

numerator_NSE_sum = dff["numerator_NSE"].sum()
numerator_Rsquared_sum = pow(dff["numerator_Rsquared"].sum(),2)
denominator_NSE_sum = dff["denominator_NSE"].sum()
denominator_2_sum = dff["denominator_2"].sum()
dff.head(5)


In [ ]:
plt.figure(figsize=(100,20))
plt.scatter(df.index, df['O2'])
plt.figure(figsize=(100,20))
plt.scatter(df['Datum'], df['Conduct'])
plt.figure(figsize=(100,20))
plt.scatter(df3['Datum'], df3['Conduct'])